In [2]:

import os 
import cv2
from tqdm import tqdm
import numpy as np
import pandas as pd
from dataclasses import dataclass

@dataclass
class Area:
    name: str
    top_x: int
    top_y: int
    bottom_x: int
    bottom_y: int


In [3]:
minimap = Area('minimap',15,620,435,1040)
left_team = Area('left_team',440,980,870,1080)
right_team = Area('right_team',1050,980,1480,1080)
timer = Area('timer',870,980,1050,1080)
kill_log = Area('kill_log',1490,780,1920,1020)
hp = Area('hp',520,0,685,100)
skill = Area('skill',745,0,1175,100)
amo = Area('amo',1235,0,1400,100)
center = ('center',960,540,125)

def get_roi(x, y):
    if ((x - center[1])**2 + (y - center[2])**2) <= center[3]**2:
        return center[0]
    elif left_team.top_x <= x <= left_team.bottom_x and left_team.top_y <= y <= left_team.bottom_y:
        return left_team.name
    elif right_team.top_x <= x <= right_team.bottom_x and right_team.top_y <= y <= right_team.bottom_y:
        return right_team.name
    elif timer.top_x <= x <= timer.bottom_x and timer.top_y <= y <= timer.bottom_y:
        return timer.name
    elif kill_log.top_x <= x <= kill_log.bottom_x and kill_log.top_y <= y <= kill_log.bottom_y:
        return kill_log.name
    elif hp.top_x <= x <= hp.bottom_x and hp.top_y <= y <= hp.bottom_y:
        return hp.name
    elif skill.top_x <= x <= skill.bottom_x and skill.top_y <= y <= skill.bottom_y:
        return skill.name
    elif amo.top_x <= x <= amo.bottom_x and amo.top_y <= y <= amo.bottom_y:
        return amo.name
    elif minimap.top_x <= x <= minimap.bottom_x and minimap.top_y <= y <= minimap.bottom_y:
        return minimap.name
    else:
        return 'other'

In [4]:
get_roi(300,1000)

'minimap'

In [5]:
get_roi(450,1000)

'left_team'

In [6]:
get_roi(880,1000)

'timer'

In [7]:
get_roi(1500,1000)

'kill_log'

In [8]:
get_roi(540,10)

'hp'

In [9]:
get_roi(750,10)

'skill'

In [11]:
get_roi(1240,10)

'amo'

In [3]:
from moviepy.editor import VideoFileClip
import os

def make_displayonly_video(video_path:str, output_dir:str='displayonly_video'):
    clip = VideoFileClip(video_path)
    cropped_clip = clip.crop(y1=0, y2=clip.h // 2)  # cropping the upper half of the video
    
    os.makedirs(output_dir, exist_ok=True)  # create the directory if it doesn't exist
    
    output_path = os.path.join(output_dir, os.path.basename(video_path))
    base_name, ext = os.path.splitext(output_path)
    i = 1
    while os.path.exists(output_path):
        print(f"{output_path} already exists, saving as {base_name}{i}.mp4 instead.")
        output_path = f"{base_name}{i}.mp4"
        i += 1

    cropped_clip.write_videofile(output_path, codec='libx264')


In [4]:
import os 
make_displayonly_video(os.path.normpath(r'data\video\x-test.mkv'))

Moviepy - Building video displayonly_video\x-test.mkv.
MoviePy - Writing audio in x-testTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video displayonly_video\x-test.mkv



Moviepy - Done !
Moviepy - video ready displayonly_video\x-test.mkv


In [1]:
from get_eyedata import ow2
import os 

In [2]:
print(ow2.get_roi(200,150))
print(ow2.get_roi(950,150))
print(ow2.get_roi(200,370))
print(ow2.get_roi(920,540))
print(ow2.get_roi(1900,1000))
print(ow2.get_roi(950,980))

hp
other
other
center
other
other


In [5]:
ow2.make_dataset(os.path.normpath(r'data\video\x-test.mkv'))

total_frames:141


100%|████████████| 141/141 [00:02<00:00, 67.23it/s]


displayonly_video\x-test.mkv already exists, saving as displayonly_video\x-test1.mp4 instead.
displayonly_video\x-test1.mp4 already exists, saving as displayonly_video\x-test2.mp4 instead.
displayonly_video\x-test2.mp4 already exists, saving as displayonly_video\x-test3.mp4 instead.
displayonly_video\x-test3.mp4 already exists, saving as displayonly_video\x-test4.mp4 instead.
displayonly_video\x-test4.mp4 already exists, saving as displayonly_video\x-test5.mp4 instead.
displayonly_video\x-test5.mp4 already exists, saving as displayonly_video\x-test6.mp4 instead.
displayonly_video\x-test6.mp4 already exists, saving as displayonly_video\x-test7.mp4 instead.
displayonly_video\x-test7.mp4 already exists, saving as displayonly_video\x-test8.mp4 instead.
Moviepy - Building video displayonly_video\x-test8.mp4.
MoviePy - Writing audio in x-test8TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video displayonly_video\x-test8.mp4



Moviepy - Done !
Moviepy - video ready displayonly_video\x-test8.mp4


,frame_ids,x,y,roi
0,0,67,886,other
1,1,67,886,other
2,2,138,992,other
3,3,138,992,other
4,4,138,992,other
...,...,...,...,...
136,136,1071,1080,match_status
137,137,1071,1080,match_status
138,138,1071,1080,match_status
139,139,1071,1080,match_status


In [4]:

import cv2
import numpy as np
import pandas as pd
from dataclasses import dataclass
from queue import Queue
from threading import Thread
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import os

@dataclass
class Area:
    name: str
    top_x: int
    top_y: int
    bottom_x: int
    bottom_y: int
    
areas = [
    Area('hp', 50, 50, 500, 250),
    Area('chat', 50, 330, 500, 500),
    Area('ult', 880, 50, 1040, 220),
    Area('my_status', 1450, 50, 1900, 280),
    Area('match_status', 650, 880, 1300, 1080),
    Area('kill_log', 1450, 880, 1920, 1080)
]

def get_roi(x, y):
    if ((x - 960)**2 + (y - 540)**2) <= 125**2:
        return 'center'
    else:
        for area in areas:
            if area.top_x <= x <= area.bottom_x and area.top_y <= y <= area.bottom_y:
                return area.name
    return 'other'

In [5]:
print(get_roi(200,150))
print(get_roi(950,150))
print(get_roi(200,370))
print(get_roi(920,540))
print(get_roi(1900,1000))
print(get_roi(950,980))

hp
ult
chat
center
kill_log
match_status


In [3]:
valo.make_dataset_from_folder('data/video')

100%|██████| 3/3 [00:15<00:00,  5.31s/it]
